#Outbrain Click Prediction 
The purpose of the Outbrain competition is to predict which recommended content each user will click. Outbrain has provided various relational datasets that need to be explored before any modeling can begin. 

###Import Libraries

###Raw Data 

In [117]:
import pandas as pd
import numpy as np
import os
import gc
import sys
import sqlite3 as sql
import matplotlib.pyplot as plt
%matplotlib inline

print('Files To Explore:')
for f in os.listdir('inputs/'):
    if 'csv' in f:
        print(f.ljust(30) + str(round(os.path.getsize('inputs/' + f)/1000000.0, 2)) + " MB")

Files To Explore:
clicks_test.csv               506.95 MB
clicks_train.csv              1486.73 MB
documents_categories.csv      118.02 MB
documents_entities.csv        324.1 MB
documents_meta.csv            89.38 MB
documents_topics.csv          339.47 MB
events.csv                    1208.55 MB
page_views_sample.csv         454.35 MB
promoted_content.csv          13.89 MB
sample_submission.csv         273.14 MB


####What are the dimensions of each dataset? Are there any common variables?

In [93]:
try:del promoted_content
except:pass;gc.collect()


In [95]:
train=pd.read_csv('inputs/clicks_train.csv')
#test=pd.read_csv('inputs/clicks_test.csv')
doc_cat=pd.read_csv('inputs/documents_categories.csv')
doc_ent=pd.read_csv('inputs/documents_entities.csv')
doc_meta=pd.read_csv('inputs/documents_meta.csv')
doc_topics=pd.read_csv('inputs/documents_topics.csv')
events=pd.read_csv('inputs/events.csv')
pg_views_samp=pd.read_csv('inputs/page_views_sample.csv')
promoted_content=pd.read_csv('inputs/promoted_content.csv')


In [106]:
dfs=[train, doc_cat,doc_ent]

In [116]:
for f in dfs:
        print('Shape:' + str(f.shape))

Shape:(87141731, 3)
Shape:(5481475, 3)
Shape:(5537552, 3)


###Review *Clicks* Data

In [79]:
print('Train Shape:' + str(train.shape))
train.head()

Train Shape:(87141731, 3)


,display_id,ad_id,clicked
0,1,42337,0
1,1,139684,0
2,1,144739,1
3,1,156824,0
4,1,279295,0


In [78]:
print('Test Shape:'+str(test.shape))
test.head()

Test Shape:(32225162, 2)


,display_id,ad_id
0,16874594,66758
1,16874594,150083
2,16874594,162754
3,16874594,170392
4,16874594,172888


In [41]:
doc_cat=pd.read_csv('inputs/documents_categories.csv')
doc_ent=pd.read_csv('inputs/documents_entities.csv')
doc_meta=pd.read_csv('inputs/documents_meta.csv')
doc_topics=pd.read_csv('inputs/documents_topics.csv')

In [49]:
doc_cat.head()

,document_id,category_id,confidence_level
0,1595802,1611,0.92
1,1595802,1610,0.07
2,1524246,1807,0.92
3,1524246,1608,0.07
4,1617787,1807,0.92


In [50]:
doc_meta.head()

,document_id,source_id,publisher_id,publish_time
0,1595802,1,603,2016-06-05 00:00:00
1,1524246,1,603,2016-05-26 11:00:00
2,1617787,1,603,2016-05-27 00:00:00
3,1615583,1,603,2016-06-07 00:00:00
4,1615460,1,603,2016-06-20 00:00:00
